In [ ]:
import yfinance as yf
import pandas as pd 
import numpy as np 
import ta
import matplotlib.pyplot as plt

In [ ]:
class Backtest:
    
    def __init__(self, symbol):
        self.symbol = symbol
        self.df = yf.download(self.symbol, start='2019-01-01')
        if self.df.empty:
            print('No data pulled')

        else:
            self.calc_indicators()
            self.generate_signals()
            self.loop_it()
            # self.profit = self.calc_profit()

    def calc_indicators(self):
        self.df['ma_20'] = self.df.Close.rolling(20).mean()
        self.df['vol'] = self.df.Close.rolling(20).std()
        self.df['upper_bb'] = self.df.ma_20 + (2 * self.df.vol)
        self.df['lower_bb'] = self.df.ma_20 - (2 * self.df.vol)
        self.df['rsi'] = ta.momentum.rsi(self.df.Close, window=6)
        self.df.dropna(inplace=True)
        
    def generate_signals(self):

        conditions = [(self.df.rsi < 30) & (self.df.Close < self.df.lower_bb),
                      (self.df.rsi > 70) & (self.df.Close > self.df.upper_bb)]
        choices = ['Buy', 'Sell']
        self.df['signal'] = np.select(conditions, choices)
        self.df.signal = self.df.signal.shift()
        self.df.dropna(inplace=True)
        
    def loop_it(self):
        position = False
        buydates, selldates = [], []

        for index, row in self.df.iterrows():
            if not position and row['signal'] == 'Buy':
                position = True
                buydates.append(index)

            if position and row['signal'] == 'Sell':
                position = False
                selldates.append(index)

        self.buy_arr = self.df.loc[buydates].Open
        self.sell_arr = self.df.loc[selldates].Open
    
    def calc_profit(self):
        if self.buy_arr[-1] > self.sell_arr[-1]:
            self.buy_arr = self.buy_arr[:-1]
            self.sell_arr = self.sell_arr[1:]

        return (self.sell_arr.values - self.buy_arr.values)/self.buy_arr.values
        

In [ ]:
symbol = 'AAPL'
instance = Backtest(symbol)

In [ ]:
df = yf.download(symbol, start='2019-01-01')
df['ma_20'] = df.Close.rolling(20).mean()
df['vol'] = df.Close.rolling(20).std()
df['upper_bb'] = df.ma_20 + (2 * df.vol)
df['lower_bb'] = df.ma_20 - (2 * df.vol)
df['rsi'] = ta.momentum.rsi(df.Close, window=6)
df.dropna(inplace=True)
conditions = [(df.rsi < 30) & (df.Close < df.lower_bb),
                (df.rsi > 70) & (df.Close > df.upper_bb)]
choices = ['Buy', 'Sell']
df['signal'] = np.select(conditions, choices)
df.signal = df.signal.shift()
df.dropna(inplace=True)

position = False
buydates, selldates = [], []

for index, row in df.iterrows():
    if not position and row['signal'] == 'Buy':
        position = True
        buydates.append(index)

    if position and row['signal'] == 'Sell':
        position = False
        selldates.append(index)

buy_arr = df.loc[buydates].Open
sell_arr = df.loc[selldates].Open

In [ ]:
df

In [ ]:
df.loc[buy_arr.index or sell_arr.index]

In [ ]:
sell_arr